## Data Ingestion

In [1]:
import pandas as pd

# column_names = ['id', 'text', 'label']
twitter_train = pd.read_csv("/kaggle/input/dataset-3/train_formatted.csv")
twitter_test = pd.read_csv("/kaggle/input/dataset-3/test_formatted.csv")
twitter_dev = pd.read_csv("/kaggle/input/dataset-3/dev_formatted.csv")

In [2]:
twitter_train.shape, twitter_test.shape, twitter_dev.shape

((1557, 5), (520, 5), (485, 5))

## Data Preparation

In [3]:
LABELS = twitter_train['label'].unique().tolist()
train_label = twitter_train['label'].replace(LABELS, [0, 1, 2, 3]).tolist()

/tmp/ipykernel_36/3616623573.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_label = twitter_train['label'].replace(LABELS, [0, 1, 2, 3]).tolist()


In [4]:
train_data = twitter_train['text'].tolist()

In [5]:
dev_data = twitter_dev['text'].tolist()
dev_label = twitter_dev['label'].replace(LABELS, [0, 1, 2, 3]).tolist()

/tmp/ipykernel_36/3987848631.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dev_label = twitter_dev['label'].replace(LABELS, [0, 1, 2, 3]).tolist()


In [6]:
test_data = twitter_test['text'].tolist()
test_label = twitter_test['label'].replace(LABELS, [0, 1, 2, 3]).tolist()

/tmp/ipykernel_36/2548705681.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_label = twitter_test['label'].replace(LABELS, [0, 1, 2, 3]).tolist()


In [7]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class ClassificationDataset(Dataset):
    def __init__(self, texts: list[str], labels: list[int], tokenizer, max_length: int):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text, 
            add_special_tokens=True, 
            max_length=self.max_length, 
            return_token_type_ids=False, 
            padding="max_length",
            truncation=True, 
            return_attention_mask=True, 
            return_tensors="pt"
        )
        
        return {
            "input_ids": encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [8]:
from transformers import BertTokenizer, BertModel, AdamW

max_length = 128

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
train_dataset = ClassificationDataset(train_data, train_label, tokenizer, max_length)
dev_dataset = ClassificationDataset(dev_data, dev_label, tokenizer, max_length)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)

In [10]:
test_dataset = ClassificationDataset(test_data, test_label, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## Model Preparation

In [11]:
num_classes = 4

In [12]:
class MBERTClassifier(nn.Module):
    def __init__(self, mbert_model, num_classes, dropout=0.1):
        super(MBERTClassifier, self).__init__()
        self.mbert = mbert_model
        self.dropout = nn.Dropout(dropout)
        self.ffn = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.mbert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.ffn(x)
        return logits

In [13]:
mbert_model = BertModel.from_pretrained('bert-base-multilingual-cased')

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [14]:
model = MBERTClassifier(mbert_model, num_classes)

In [15]:
learning_rate = 2e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

MBERTClassifier(
  (mbert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
from tqdm import tqdm

def calculate_metrics(y_true, y_pred, y_pred_proba, num_classes):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted'),
        'recall': recall_score(y_true, y_pred, average='weighted'),
        'f1': f1_score(y_true, y_pred, average='weighted')
    }
    
    # Calculate ROC-AUC score
    if num_classes == 2:
        metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba[:, 1])
    else:
        try:
            metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba, multi_class='ovr', average='weighted')
        except:
            metrics['roc_auc'] = None
            
    return metrics

In [17]:
def evaluate(model, data_loader, device, num_classes):
    model.eval()
    all_labels = []
    all_predictions = []
    all_predictions_proba = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            all_predictions_proba.extend(probabilities.cpu().numpy())

    all_labels = np.array(all_labels)
    all_predictions = np.array(all_predictions)
    all_predictions_proba = np.array(all_predictions_proba)

    return calculate_metrics(all_labels, all_predictions, all_predictions_proba, num_classes)

In [18]:
def train(model, train_loader, val_loader, criterion, optimizer, device, num_epochs):
    best_val_metrics = {'f1': 0.0}
    history = {'train': [], 'val': []}

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        all_train_labels = []
        all_train_predictions = []
        all_train_predictions_proba = []

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_train_labels.extend(labels.cpu().numpy())
            all_train_predictions.extend(predicted.cpu().numpy())
            all_train_predictions_proba.extend(probabilities.cpu().detach().numpy())

        # Calculate training metrics
        all_train_labels = np.array(all_train_labels)
        all_train_predictions = np.array(all_train_predictions)
        all_train_predictions_proba = np.array(all_train_predictions_proba)
        train_metrics = calculate_metrics(all_train_labels, all_train_predictions, 
                                       all_train_predictions_proba, num_classes)
        
        # Validation phase
        val_metrics = evaluate(model, val_loader, device, num_classes)
        
        # Store metrics history
        history['train'].append({
            'loss': train_loss / len(train_loader),
            **train_metrics
        })
        history['val'].append(val_metrics)

        # Print epoch results
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("Training Metrics:")
        print(f"Loss: {train_loss/len(train_loader):.4f}")
        for metric, value in train_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
        
        print("\nValidation Metrics:")
        for metric, value in val_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
    
    return history

In [19]:
num_epochs = 5

optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
history = train(model, train_loader, dev_loader, criterion, optimizer, device, num_epochs)

Epoch 1/5: 100%|██████████| 49/49 [00:19<00:00,  2.49it/s]
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Epoch 1/5
Training Metrics:
Loss: 1.3879
Accuracy: 0.2620
Precision: 0.2648
Recall: 0.2620
F1: 0.2607
Roc_auc: 0.5106

Validation Metrics:
Accuracy: 0.2515
Precision: 0.0633
Recall: 0.2515
F1: 0.1011
Roc_auc: 0.5632


Epoch 2/5: 100%|██████████| 49/49 [00:18<00:00,  2.60it/s]



Epoch 2/5
Training Metrics:
Loss: 1.3732
Accuracy: 0.3006
Precision: 0.2884
Recall: 0.3006
F1: 0.2699
Roc_auc: 0.5677

Validation Metrics:
Accuracy: 0.4062
Precision: 0.3756
Recall: 0.4062
F1: 0.3568
Roc_auc: 0.6257


Epoch 3/5: 100%|██████████| 49/49 [00:18<00:00,  2.60it/s]



Epoch 3/5
Training Metrics:
Loss: 1.2827
Accuracy: 0.4297
Precision: 0.4188
Recall: 0.4297
F1: 0.3993
Roc_auc: 0.6859

Validation Metrics:
Accuracy: 0.4763
Precision: 0.5102
Recall: 0.4763
F1: 0.4416
Roc_auc: 0.7163


Epoch 4/5: 100%|██████████| 49/49 [00:18<00:00,  2.60it/s]



Epoch 4/5
Training Metrics:
Loss: 1.0472
Accuracy: 0.5947
Precision: 0.5934
Recall: 0.5947
F1: 0.5779
Roc_auc: 0.8225

Validation Metrics:
Accuracy: 0.5691
Precision: 0.5682
Recall: 0.5691
F1: 0.5660
Roc_auc: 0.7963


Epoch 5/5: 100%|██████████| 49/49 [00:18<00:00,  2.60it/s]



Epoch 5/5
Training Metrics:
Loss: 0.7168
Accuracy: 0.7643
Precision: 0.7621
Recall: 0.7643
F1: 0.7618
Roc_auc: 0.9200

Validation Metrics:
Accuracy: 0.6144
Precision: 0.6360
Recall: 0.6144
F1: 0.6151
Roc_auc: 0.8294


In [21]:
import pandas as pd

pd.DataFrame(history['train'])

,loss,accuracy,precision,recall,f1,roc_auc
0,1.387883,0.262042,0.264814,0.262042,0.260735,0.510568
1,1.373165,0.300578,0.288378,0.300578,0.269883,0.567732
2,1.282684,0.429672,0.418768,0.429672,0.399336,0.685895
3,1.047210,0.594733,0.593426,0.594733,0.577946,0.822456
4,0.716793,0.764290,0.762149,0.764290,0.761823,0.919987


In [22]:
pd.DataFrame(history['val'])

,accuracy,precision,recall,f1,roc_auc
0,0.251546,0.063276,0.251546,0.101116,0.563208
1,0.406186,0.375625,0.406186,0.356816,0.625734
2,0.476289,0.510181,0.476289,0.441561,0.716287
3,0.569072,0.568184,0.569072,0.566050,0.796253
4,0.614433,0.635978,0.614433,0.615096,0.829423


In [23]:
evaluate(model, test_loader, device, num_classes)

{'accuracy': 0.6153846153846154,
 'precision': 0.6369555568573407,
 'recall': 0.6153846153846154,
 'f1': 0.6155114885344032,
 'roc_auc': 0.8289832740647366}